# Nvidia GPU INT-8 quantization on any transformers model (encoder based)

Quantization is one of the most effective and generic approach to make model inference faster.
Basically, it replaces high precision float numbers in model tensors encoded in 32 or 16 bits by lower precision ones encoded in 8 bits or less:

* it takes less memory
* computation is easier / faster

It can be applied to any model in theory, and, if done well, it should not decrease its accuracy.

The purpose of this notebook is to show 2 processes to perform quantization on any `transformer` architectures.

**TL;DR, we benchmarked Pytorch and Nvidia TensorRT, on both CPU and GPU, with/without quantization, our methods provide the fastest inference by large margin**.

| Framework                 | Precision | Latency (ms) | Accuracy | Speedup    | Hardware |
|:--------------------------|-----------|--------------|----------|:-----------|:--------:|
| Pytorch                   | FP32      | 4000         | 86.8 %   | X 0.02     |   CPU    |
| Pytorch                   | FP16      | 4005         | 86.8 %   | X 0.02     |   CPU    |
| Pytorch                   | **INT-8** | 3670         | 86.8 %   | X 0.02     | **CPU**  |
| Pytorch                   | FP32      | 80           | 86.8 %   | X 1        |   GPU    |
| Pytorch                   | FP16      | 58           | 86.8 %   | X 1.38     |   GPU    |
| ONNX Runtime              | FP32      | 74           | 86.8 %   | X 1.08     |   GPU    |
| ONNX Runtime              | FP16      | 34           | 86.8 %   | X 2.35     |   GPU    |
| ONNX Runtime              | FP32      | 3767         | 86.8 %   | X 0.02     |   CPU    |
| ONNX Runtime              | FP16      | 4607         | 86.8 %   | X 0.02     |   CPU    |
| ONNX Runtime              | **INT-8** | 3712         | 86.8 %   | X 0.02     | **CPU**  |
| TensorRT                  | FP16      | 30           | 86.8 %   | X 2.67     |   GPU    |
| TensorRT (**our method**) | **INT-8** | 15           | 84.4 %   | **X 5.33** | **GPU**  |

> measures done on a Nvidia RTX 3090 GPU + 12 cores i7 Intel CPU (support AVX-2 instructions)
>
> `base` architecture flavor with batch of size 32 / seq len 256, similar results obtained for other sizes/seq len not included in the table.
>
> accuracy obtained after a single epoch, no LR search or any hyper parameter optimization


## A (very) short intro to INT-8 quantization

Basic idea behind model quantization is to replace tensors made of float numbers (usually encoded on 32 bits) by lower precision representation (integers encoded on 8 bits for Nvidia GPUs).
Therefore computation is faster and model memory footprint is lower. Making tensor storage smaller makes memory transfer faster... and is also a source of computation acceleration.
This technic is very interesting for its trade-off: you reduce inference time significantly, and when dataset is large enough, it costs close to nothing in accuracy.

Replacing float numbers by integers is done through a mapping.
This step is called `calibration`, and its purpose is to compute for each tensor or each channel of a tensor (one of its dimensions) a range of all possible values and then define a scale and a distribution center to map float numbers to 8 bits integers.

There are several ways to perform quantization, depending of how and when the `calibration` is performed:

* dynamically: the mapping is done during the inference, there are some overhead but it's easy to put in place and usually the accuracy is preserved,
* statically, after training (`post training quantization` or `PTQ`): this way is efficient, but it may have a significant accuracy cost,
* statically, before training (`quantization aware training` or `QAT`): this way is efficient and has a low accuracy cost as the weights will take care of the result

Nvidia GPUs don't support dynamic quantization, CPU supports all type of quantization.  
Compared to `PTQ`, `QAT` better presevers accuracy and should be prefered.


During the quantization aware *training*:

* in the inside, Pytorch will train with high precision float numbers,
* on the outside, Pytorch will simulate that a quantization has already been applied and output results accordingly (for loss computation for instance)

The simulation process is done through the add of quantization / dequantization nodes, most often called `QDQ`, it's an abbreviation you will see often in quantization world.



> Want to learn more about quantization?
> 
> * You can check this [high quality blog post](https://leimao.github.io/article/Neural-Networks-Quantization/) for more information.
> * The process is well described in this [Nvidia presentation](https://on-demand.gputechconf.com/gtc/2017/presentation/s7310-8-bit-inference-with-tensorrt.pdf)

## Why this notebook?

CPU quantization is supported out of the box by `Pytorch` and `ONNX Runtime`.
**GPU quantization on the other side requires specific tools and process to be applied**.

In the specific case of `transformer` models, until recently (december 2021), the only way shown by Nvidia is to build manually the graph of our models in `TensorRT`. This is a low level approach, based on GPU capacity knowledge (which operators are supported, etc.). It's certainly out of reach of most NLP practitioners and is very time consuming to update/adapt to new architectures.

Hopefully, Nvidia added to Hugging Face `transformer` library a new model called `QDQBert` few weeks ago.
Basically, it's a vanilla `Bert` architecture which supports INT-8 quantization.
It doesn't support moedern architectures out of the box, like `Albert`, `Roberta`, `deberta` or `Electra`.
Nvidia also provide a demo dedicated to the SQuaD task.

This open the door to extension of the approach to other architectures.

To be both simple and cover most use cases, in this notebook we will see:

* how to perform GPU quantization on **any** transformer model (not just Bert)
* how to apply quantization to a common task like classification (which is easier to understand than question answering)
* measure performance gain (latency)


## Project setup

### Dependencies installation

We install `master` branch of `transfomers` library to use a new model: **QDQBert** and `transformer-deploy` to leverage `TensorRT` models (TensorRT API is not something simple to master, it's highly advised to use a wrapper). Your machine should have Nvidia CUDA 11.X, TensorRT 8.2.1 and cuBLAS installed. It's said to be tricky to install, in my experience, just follow Nvidia instructions **and nothing else**, it should work out of the box. Docker image with TensorRT 8.2.1 has not yet been released, this notebook will be updated when it's ready.

In [1]:
#! pip3 install transformers datasets sklearn
#! pip3 install git+ssh://git@github.com/ELS-RD/transformer-deploy
#! pip3 install git+ssh://git@github.com/NVIDIA/TensorRT#egg=pytorch-quantization\&subdirectory=tools/pytorch-quantization/

Check the GPU is enabled and usable.

In [2]:
! nvidia-smi

Tue Dec 28 13:48:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 495.44       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:03:00.0  On |                  N/A |
| 35%   41C    P8    28W / 350W |    254MiB / 24267MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
import logging
import os
from collections import OrderedDict
from typing import Dict, List
from typing import OrderedDict as OD
from typing import Union

import datasets
import numpy as np
import pycuda.autoinit
import tensorrt as trt
import torch
import transformers
from datasets import load_dataset, load_metric
from pycuda._driver import Stream
from tensorrt.tensorrt import IExecutionContext, Logger, Runtime
from pytorch_quantization import nn as quant_nn

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    IntervalStrategy,
    PreTrainedModel,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
)

from transformer_deploy.backends.ort_utils import (
    convert_to_onnx,
    convert_to_quant_onnx,
    cpu_quantization,
    create_model_for_provider,
    optimize_onnx,
)
from transformer_deploy.backends.trt_utils import build_engine, get_binding_idxs, infer_tensorrt
from transformer_deploy.benchmarks.utils import print_timings, track_infer_time
from transformer_deploy.QDQModels.calibration_utils import QATCalibrate

Set logging to `error` level to ease readability of this `notebook` on Github.

In [4]:
log_level = logging.ERROR
logging.getLogger().setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
transformers.logging.set_verbosity_error()

### Preprocess data

This part is inspired from an [official Notebooks from Hugging Face](https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb).

There is nothing special to do. Define the task:

In [5]:
model_name = "roberta-base"
task = "mnli"
num_labels = 3
batch_size = 32
max_seq_len = 256
validation_key = "validation_matched"
timings: Dict[str, List[float]] = dict()
runtime: Runtime = trt.Runtime(trt_logger)
profile_index = 0

Preprocess data (task specific):

In [6]:
def preprocess_function(examples):
    return tokenizer(
        examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=max_seq_len
    )


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


def convert_tensor(data: OD[str, List[List[int]]], output: str) -> OD[str, Union[np.ndarray, torch.Tensor]]:
    input: OD[str, Union[np.ndarray, torch.Tensor]] = OrderedDict()
    for k in ["input_ids", "attention_mask", "token_type_ids"]:
        if k in data:
            v = data[k]
            if output == "torch":
                value = torch.tensor(v, dtype=torch.long, device="cuda")
            elif output == "np":
                value = np.asarray(v, dtype=np.int32)
            else:
                raise Exception(f"unknown output type: {output}")
            input[k] = value
    return input


def measure_accuracy(infer, int64: bool) -> float:
    outputs = list()
    for start_index in range(0, len(encoded_dataset[validation_key]), batch_size):
        end_index = start_index + batch_size
        data = encoded_dataset[validation_key][start_index:end_index]
        inputs: OD[str, np.ndarray] = convert_tensor(data=data, output="np")
        if int64:
            for k, v in inputs.items():
                inputs[k] = v.astype(np.int64)
        output = infer(inputs)
        output = np.argmax(output[0], axis=1).astype(int).tolist()
        outputs.extend(output)
    return np.mean(np.array(outputs) == np.array(validation_labels))


def get_trainer(model: PreTrainedModel) -> Trainer:
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset[validation_key],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    transformers.logging.set_verbosity_error()
    return trainer

In [7]:
tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

dataset = load_dataset("glue", task)
metric = load_metric("glue", task)

encoded_dataset = dataset.map(preprocess_function, batched=True)
validation_labels = [item["label"] for item in encoded_dataset[validation_key]]

nb_step = 1000
strategy = IntervalStrategy.STEPS
args = TrainingArguments(
    f"{model_name}-{task}",
    evaluation_strategy=strategy,
    eval_steps=nb_step,
    logging_steps=nb_step,
    save_steps=nb_step,
    save_strategy=strategy,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size * 2,
    num_train_epochs=1,
    fp16=True,
    group_by_length=True,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[],
)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

## (Standard) fine-tuning model

Now that our data are ready, we can download/fine tune the pretrained model.

In [9]:
model_fp16: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
trainer = get_trainer(model_fp16)
transformers.logging.set_verbosity_error()
trainer.train()
print(trainer.evaluate())

model_fp16.save_pretrained("model_trained_fp16")

[INFO|trainer.py:439] 2021-12-27 09:19:51,063 >> Using amp half precision backend


{'loss': 0.6605, 'learning_rate': 9.1875814863103e-06, 'epoch': 0.08}
{'eval_loss': 0.4653007388114929, 'eval_accuracy': 0.8183392766174223, 'eval_runtime': 18.2981, 'eval_samples_per_second': 536.393, 'eval_steps_per_second': 8.416, 'epoch': 0.08}
{'loss': 0.4956, 'learning_rate': 8.372718383311604e-06, 'epoch': 0.16}
{'eval_loss': 0.4208127558231354, 'eval_accuracy': 0.8346408558329088, 'eval_runtime': 18.3709, 'eval_samples_per_second': 534.268, 'eval_steps_per_second': 8.383, 'epoch': 0.16}
{'loss': 0.4662, 'learning_rate': 7.557855280312908e-06, 'epoch': 0.24}
{'eval_loss': 0.42171549797058105, 'eval_accuracy': 0.8358634742740703, 'eval_runtime': 18.3642, 'eval_samples_per_second': 534.464, 'eval_steps_per_second': 8.386, 'epoch': 0.24}
{'loss': 0.4458, 'learning_rate': 6.7429921773142115e-06, 'epoch': 0.33}
{'eval_loss': 0.3808833658695221, 'eval_accuracy': 0.8527763627101376, 'eval_runtime': 18.3578, 'eval_samples_per_second': 534.649, 'eval_steps_per_second': 8.389, 'epoch': 0.

## Add quantization support to any model

The idea is to take the source code of a specific model and add automatically `QDQ` nodes. That way, quantization will work out of the box for this architecture.
The process is based on Python AST modification, basically we parse the source code in RAM, we convert it to a tree, then we patch the tree to add the QDQ nodes and we replace, in RAM, the original module source code.

In theory it works for any model. However, not related to quantization, some models are not fully compliant with `TensorRT` (unsupported operators, etc.).
For those models, we rewrite some part of the source code, these patch are manually written but are applied to model at run time (like the AST manipulation).

> concrete examples on `Roberta` architecture: in HF library, there is a `cumsum` in the position embedding generation. Something very simple. It takes as input an integer tensor and output an integer tensor. It happens that the `cumsum` operator from TensorRT supports float but not integer (https://github.com/onnx/onnx-tensorrt/blob/master/docs/operators.md). It leads to a crash during the model conversion with a strange error message. Converting the input to float tensor fix the issue. Not complex, but requires some knowledge.

The process below is:

* Calibrate
* Quantization Aware training (QAT)

> there are many ways to get a QDQ model, you can modify Pytorch source code (including doing it at runtime like here), patch ONNX graph (this approach is used at Microsoft for instance but only support PTQ, not QAT as ONNX file can't be trained on Pytorch for now) or leverage the new FX Pytorch interface (it's a bit experimental and it seems to miss some feature to support Nvida QAT library). Modifying the source code is the most straight forward so we choosed to do it that way.


### Post Training Quantization (PTQ)

A PTQ is basically a finetuned model where we add quantization nodes and that we calibrate.

Calibration is a key step in the static quantization process. On its quality depends the final accuracy (the inference speed will stay the same).  
Moreover, a good PTQ is a good basis for a good Quantization Aware Training (QAT).

By calling `with QATCalibrate(...) as qat:`, the lib will patch transformer model AST (source code) in RAM, basically adding quantization support to each model.

#### Calibration percentile grid search

One of the thing we try to guess during the calibration is what range of tensor values capture most of the information stored in the model. Indeed, a FP32 tensor can store at the same time very large and very small values, we obviously can't do the same with a 8-bits integer tensors and a scale. 8-bits integer can only encode 255 values so we need to fix some limits and say, if a value is outside our limits, it just takes a maximum value instead of its real one. For instance, if we say our range is -1000 to +1000 and a tensor contain the value +4000, it will be replaced by the maximum value, +1000.

As said before, we will use the histogram method to find the perfect range. We also need to choose a percentile. Usually, you will choose something very close to 100.

If the percentile is too small, we put too many values outside the final range. Values outside the range will be replaced by a single maximum value and you loose some granularity in model weights.

If the percetile is too big, your range will be very large and because 8-bits signed integers can only encode values between -127 to +127, you loose in granularity.

Therefore, we launch a grid search on percentile hyper parameter.

In [19]:
for percentile in [99.9, 99.99, 99.999, 99.9999]:
    with QATCalibrate(method="histogram", percentile=percentile) as qat:
        model_q: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
            "model_trained_fp16", num_labels=num_labels
        )
        model_q = model_q.cuda()
        qat.setup_model_qat(model_q)  # prepare quantizer to any model

        with torch.no_grad():
            for start_index in range(0, 128, batch_size):
                end_index = start_index + batch_size
                data = encoded_dataset["train"][start_index:end_index]
                input_torch = {
                    k: torch.tensor(v, dtype=torch.long, device="cuda")
                    for k, v in data.items()
                    if k in ["input_ids", "attention_mask", "token_type_ids"]
                }
                model_q(**input_torch)
    trainer = get_trainer(model_q)
    print(f"percentile: {percentile}")
    print(trainer.evaluate())

[INFO|trainer.py:439] 2021-12-27 17:25:51,070 >> Using amp half precision backend


percentile: 99.9
{'eval_loss': 0.47421666979789734, 'eval_accuracy': 0.8121242995415181, 'eval_runtime': 47.9158, 'eval_samples_per_second': 204.839, 'eval_steps_per_second': 3.214}
{'eval_loss': 0.47421666979789734, 'eval_accuracy': 0.8121242995415181, 'eval_runtime': 47.9158, 'eval_samples_per_second': 204.839, 'eval_steps_per_second': 3.214}


[INFO|trainer.py:439] 2021-12-27 17:30:13,795 >> Using amp half precision backend


percentile: 99.99
{'eval_loss': 0.3841923773288727, 'eval_accuracy': 0.8487009679062659, 'eval_runtime': 46.6715, 'eval_samples_per_second': 210.3, 'eval_steps_per_second': 3.3}
{'eval_loss': 0.3841923773288727, 'eval_accuracy': 0.8487009679062659, 'eval_runtime': 46.6715, 'eval_samples_per_second': 210.3, 'eval_steps_per_second': 3.3}


[INFO|trainer.py:439] 2021-12-27 17:34:34,280 >> Using amp half precision backend


percentile: 99.999
{'eval_loss': 0.3939284086227417, 'eval_accuracy': 0.850636780438105, 'eval_runtime': 49.1138, 'eval_samples_per_second': 199.842, 'eval_steps_per_second': 3.136}
{'eval_loss': 0.3939284086227417, 'eval_accuracy': 0.850636780438105, 'eval_runtime': 49.1138, 'eval_samples_per_second': 199.842, 'eval_steps_per_second': 3.136}


[INFO|trainer.py:439] 2021-12-27 17:38:54,289 >> Using amp half precision backend


percentile: 99.9999
{'eval_loss': 1.0285985469818115, 'eval_accuracy': 0.4956698930208864, 'eval_runtime': 48.0849, 'eval_samples_per_second': 204.118, 'eval_steps_per_second': 3.203}
{'eval_loss': 1.0285985469818115, 'eval_accuracy': 0.4956698930208864, 'eval_runtime': 48.0849, 'eval_samples_per_second': 204.118, 'eval_steps_per_second': 3.203}


As you can see, the chosen percentile value has a high impact on the final accuracy.

For the rest of the notebook, we apply the `99.999` percentile.

In [8]:
with QATCalibrate(method="histogram", percentile=99.999) as qat:
    model_q: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
        "model_trained_fp16", num_labels=num_labels
    )
    model_q = model_q.cuda()
    qat.setup_model_qat(model_q)  # prepare quantizer to any model

    with torch.no_grad():
        for start_index in range(0, 128, batch_size):
            end_index = start_index + batch_size
            data = encoded_dataset["train"][start_index:end_index]
            input_torch = {
                k: torch.tensor(v, dtype=torch.long, device="cuda")
                for k, v in data.items()
                if k in ["input_ids", "attention_mask", "token_type_ids"]
            }
            model_q(**input_torch)
trainer = get_trainer(model_q)
print(trainer.evaluate())

[INFO|trainer.py:439] 2021-12-28 13:52:09,215 >> Using amp half precision backend


{'eval_loss': 0.3939284086227417, 'eval_accuracy': 0.850636780438105, 'eval_runtime': 46.5572, 'eval_samples_per_second': 210.816, 'eval_steps_per_second': 3.308}
{'eval_loss': 0.3939284086227417, 'eval_accuracy': 0.850636780438105, 'eval_runtime': 46.5572, 'eval_samples_per_second': 210.816, 'eval_steps_per_second': 3.308}


#### Per layer quantization analysis

Below we will run a sensitivity analysis, by enabling quantization of one layer at a time and measuring the accuracy. That way we will be able to detect if the quantization of a specific layer has a larger cost on accuracy than other layers.

In [10]:
from pytorch_quantization import nn as quant_nn

for i in range(12):
    layer_name = f"layer.{i}"
    print(layer_name)
    for name, module in model_q.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if layer_name in name:
                module.enable_quant()
            else:
                module.disable_quant()
    trainer.evaluate()
    print("----")

layer.0
{'eval_loss': 0.35163024067878723, 'eval_accuracy': 0.8663270504330107, 'eval_runtime': 20.695, 'eval_samples_per_second': 474.27, 'eval_steps_per_second': 7.441}
----
layer.1
{'eval_loss': 0.3527306318283081, 'eval_accuracy': 0.8661232806928171, 'eval_runtime': 26.1334, 'eval_samples_per_second': 375.573, 'eval_steps_per_second': 5.893}
----
layer.2
{'eval_loss': 0.3557673394680023, 'eval_accuracy': 0.8629648497198166, 'eval_runtime': 21.1364, 'eval_samples_per_second': 464.366, 'eval_steps_per_second': 7.286}
----
layer.3
{'eval_loss': 0.3551430106163025, 'eval_accuracy': 0.8649006622516556, 'eval_runtime': 20.9252, 'eval_samples_per_second': 469.051, 'eval_steps_per_second': 7.36}
----
layer.4
{'eval_loss': 0.35053929686546326, 'eval_accuracy': 0.8649006622516556, 'eval_runtime': 21.05, 'eval_samples_per_second': 466.271, 'eval_steps_per_second': 7.316}
----
layer.5
{'eval_loss': 0.35701483488082886, 'eval_accuracy': 0.865206316861946, 'eval_runtime': 20.9236, 'eval_samples_

It seems that quantization of layers 2 to 6 has the largest accuracy impact.


#### Operator quantization analysis


Below we will run a sensitivity analysis, by enabling quantization of one operator type at a time and measuring the accuracy. That way we will be able to detect if a specific operator has a larger cost on accuracy. On Roberta we only quantize `matmul` and `LayerNorm`, so we test both candidates.

In [11]:
for op in ["matmul", "layernorm"]:
    for name, module in model_q.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if op in name:
                module.enable_quant()
            else:
                module.disable_quant()
    print(op)
    trainer.evaluate()
    print("----")

matmul
{'eval_loss': 0.35049352049827576, 'eval_accuracy': 0.8658176260825268, 'eval_runtime': 26.1972, 'eval_samples_per_second': 374.659, 'eval_steps_per_second': 5.878}
----
layernorm
{'eval_loss': 0.35847699642181396, 'eval_accuracy': 0.8597045338767193, 'eval_runtime': 24.3004, 'eval_samples_per_second': 403.903, 'eval_steps_per_second': 6.337}
----


It appears that `LayerNorm` quantization has a significant accuracy cost.

In [11]:
disable_layer_names = ["layer.2", "layer.3", "layer.4", "layer.6"]

for name, module in model_q.named_modules():
    if isinstance(module, quant_nn.TensorQuantizer):
        if any([f"{l}.output.layernorm" in name for l in disable_layer_names]):
            print(f"disable {name}")
            module.disable_quant()
        else:
            module.enable_quant()
trainer.evaluate()

disable roberta.encoder.layer.2.output.layernorm_quantizer_0
disable roberta.encoder.layer.2.output.layernorm_quantizer_1
disable roberta.encoder.layer.3.output.layernorm_quantizer_0
disable roberta.encoder.layer.3.output.layernorm_quantizer_1
disable roberta.encoder.layer.4.output.layernorm_quantizer_0
disable roberta.encoder.layer.4.output.layernorm_quantizer_1
disable roberta.encoder.layer.6.output.layernorm_quantizer_0
disable roberta.encoder.layer.6.output.layernorm_quantizer_1
{'eval_loss': 0.3660135269165039, 'eval_accuracy': 0.8618441161487519, 'eval_runtime': 45.9324, 'eval_samples_per_second': 213.684, 'eval_steps_per_second': 3.353}


{'eval_loss': 0.3660135269165039,
 'eval_accuracy': 0.8618441161487519,
 'eval_runtime': 45.9324,
 'eval_samples_per_second': 213.684,
 'eval_steps_per_second': 3.353}

By just disabling quantization for a single operator on a few layers, we keep most of the performance boost (quantization) but retrieve more than 1 point of accuracy. It's also possible to perform an analysis per quantizer to get a smaller granularity but it's a bit slow to run.

### Quantization Aware Training (QAT)

We retrain the model with 1/10 or 1/100 of the original learning rate. Our goal is to retrieve most of the original accuracy. Usually you can use a part of the train set.

In [ ]:
args.learning_rate = 1e-7
trainer = get_trainer(model_q)
trainer.train()
print(trainer.evaluate())
model_q.save_pretrained("model-qat")

[INFO|trainer.py:439] 2021-12-28 13:54:41,146 >> Using amp half precision backend


### Benchmark

#### Export a `QDQ Pytorch` model on `ONNX`, we need to enable fake quantization mode from Pytorch.

In [53]:
data = encoded_dataset["train"][1:3]
input_torch = convert_tensor(data, output="torch")
convert_to_quant_onnx(model_pytorch=model_q, output_path="model_qat.onnx", inputs_pytorch=input_torch)

In [54]:
del model_q
QATCalibrate.restore()

#### Convert `ONNX` graph to `TensorRT` engine

In [55]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="model_qat.onnx",
    logger=trt_logger,
    min_shape=(1, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=True,
    int8=True,
)

In [56]:
# same conversion from the terminal
# ,token_type_ids:32x256
# !/usr/src/tensorrt/bin/trtexec --onnx=model_qat.onnx --shapes=input_ids:32x256,attention_mask:32x256 --best --workspace=10000 --saveEngine="test.plan"

#### Prepare input and output buffer

In [57]:
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]

In [58]:
data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")
input_np: OD[str, np.ndarray] = convert_tensor(data=data, output="np")

#### Inference on `TensorRT`

We first check that inference is working correctly:

In [59]:
tensorrt_output = infer_tensorrt(
    context=context,
    host_inputs=input_np,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)
print(tensorrt_output)

[array([[ 0.3555643 ,  2.491011  , -2.3507996 ],
       [ 2.8667073 , -1.095514  , -1.5070786 ],
       [ 2.0140486 ,  0.3065021 , -1.9585495 ],
       [ 2.2055676 ,  0.29951063, -2.2425632 ],
       [ 2.7326503 , -0.18695493, -2.2924087 ],
       [ 4.1053786 , -1.0233743 , -2.7114294 ],
       [-3.583274  ,  2.672273  ,  1.2139126 ],
       [ 4.0707603 , -1.1044825 , -2.4349666 ],
       [ 0.322169  , -1.1554759 ,  0.89057773],
       [-3.345267  , -0.3590604 ,  3.3627117 ],
       [ 3.7562432 , -0.6960117 , -2.6524336 ],
       [-2.3663151 , -1.3245198 ,  3.6388462 ],
       [-2.4342794 ,  3.723619  , -0.8023841 ],
       [-2.0167181 ,  1.9331539 ,  0.21542098],
       [ 0.42074788, -0.8792107 ,  0.4193994 ],
       [ 1.4081649 , -1.0378735 , -0.01496227],
       [ 4.0923953 , -0.5934528 , -2.997319  ],
       [-2.7030241 , -2.2501273 ,  4.499137  ],
       [-3.3829408 , -0.6287503 ,  3.8455353 ],
       [-2.1994479 ,  4.069879  , -1.3168232 ],
       [ 4.007247  , -0.6379149 , -2.88

Measure of the accuracy:

In [60]:
infer_trt = lambda inputs: infer_tensorrt(
    context=context,
    host_inputs=inputs,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)

measure_accuracy(infer=infer_trt, int64=False)

0.8585838003056546

Latency measures:

In [61]:
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (INT-8)", timings=time_buffer)

[TensorRT (INT-8)] mean=17.92ms, sd=0.53ms, min=17.21ms, max=20.23ms, median=17.73ms, 95p=19.27ms, 99p=19.82ms


In [62]:
del engine, context

## Pytorch baseline

### GPU execution

We will measure vanilla Pytorch inference on both FP32 and FP16 precision, it will be our baseline:

In [8]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("model_trained_fp16", num_labels=num_labels)
baseline_model = baseline_model.cuda()
baseline_model = baseline_model.eval()

data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")

with torch.inference_mode():
    for _ in range(30):
        _ = baseline_model(**input_torch)
        torch.cuda.synchronize()
    time_buffer = list()
    for _ in range(100):
        with track_infer_time(time_buffer):
            _ = baseline_model(**input_torch)
            torch.cuda.synchronize()
print_timings(name="Pytorch (FP32)", timings=time_buffer)

[Pytorch (FP32)] mean=121.09ms, sd=1.05ms, min=120.22ms, max=125.82ms, median=120.84ms, 95p=123.41ms, 99p=124.63ms


In [ ]:
with torch.inference_mode():
    with torch.cuda.amp.autocast():
        for _ in range(30):
            _ = baseline_model(**input_torch)
            torch.cuda.synchronize()
        time_buffer = []
        for _ in range(100):
            with track_infer_time(time_buffer):
                _ = baseline_model(**input_torch)
                torch.cuda.synchronize()
print_timings(name="Pytorch (FP16)", timings=time_buffer)
del baseline_model

### CPU execution

In [ ]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("model_trained_fp16", num_labels=num_labels)
baseline_model = baseline_model.eval()
data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")
input_torch_cpu = {k: v.to("cpu") for k, v in input_torch.items()}

torch.set_num_threads(os.cpu_count())

with torch.inference_mode():
    for _ in range(3):
        _ = baseline_model(**input_torch_cpu)
        torch.cuda.synchronize()
    time_buffer = list()
    for _ in range(10):
        with track_infer_time(time_buffer):
            _ = baseline_model(**input_torch_cpu)
            torch.cuda.synchronize()
print_timings(name="Pytorch (FP32) - CPU", timings=time_buffer)

In [ ]:
with torch.inference_mode():
    with torch.cuda.amp.autocast():
        for _ in range(3):
            _ = baseline_model(**input_torch_cpu)
            torch.cuda.synchronize()
        time_buffer = []
        for _ in range(10):
            with track_infer_time(time_buffer):
                _ = baseline_model(**input_torch_cpu)
                torch.cuda.synchronize()
print_timings(name="Pytorch (FP16) - CPU", timings=time_buffer)
del baseline_model

Below, we will perform dynamic quantization on CPU.

In [ ]:
quantized_baseline_model = AutoModelForSequenceClassification.from_pretrained(
    "model_trained_fp16", num_labels=num_labels
)
quantized_baseline_model = quantized_baseline_model.eval()
quantized_baseline_model = torch.quantization.quantize_dynamic(
    quantized_baseline_model, {torch.nn.Linear}, dtype=torch.qint8
)

with torch.inference_mode():
    for _ in range(3):
        _ = quantized_baseline_model(**input_torch_cpu)
        torch.cuda.synchronize()
    time_buffer = list()
    for _ in range(10):
        with track_infer_time(time_buffer):
            _ = quantized_baseline_model(**input_torch_cpu)
            torch.cuda.synchronize()
print_timings(name="Pytorch (INT-8) - CPU", timings=time_buffer)
del quantized_baseline_model

## TensorRT baseline

Below we export our finetuned model, the purpose is to only check the performance on mixed precision (FP16, no quantization).

In [17]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("model_trained_fp16", num_labels=num_labels)
baseline_model = baseline_model.cuda()
convert_to_onnx(baseline_model, output_path="baseline.onnx", inputs_pytorch=input_torch, opset=12)
del baseline_model

In [15]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="baseline.onnx",
    logger=trt_logger,
    min_shape=(batch_size, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=True,
    int8=False,
)
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]
for _ in range(30):
    _ = infer_tensorrt(
        context=context,
        host_inputs=input_np,
        input_binding_idxs=input_binding_idxs,
        output_binding_idxs=output_binding_idxs,
        stream=stream,
    )
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (FP16)", timings=time_buffer)
del engine, context

[libprotobuf WARNING google/protobuf/io/coded_stream.cc:604] Reading dangerously large protocol message.  If the message turns out to be larger than 2147483647 bytes, parsing will be halted for security reasons.  To increase the limit (or to disable these warnings), see CodedInputStream::SetTotalBytesLimit() in google/protobuf/io/coded_stream.h.
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:81] The total number of bytes read was 556931338
Unsupported ONNX data type: UINT8 (2)


[12/26/2021-23:12:18] [TRT] [E] 4: [network.cpp::validate::2633] Error Code 4: Internal Error (Network must have at least one output)
[12/26/2021-23:12:18] [TRT] [E] 2: [builder.cpp::buildSerializedNetwork::609] Error Code 2: Internal Error (Assertion enginePtr != nullptr failed. )


TypeError: deserialize_cuda_engine(): incompatible function arguments. The following argument types are supported:
    1. (self: tensorrt.tensorrt.Runtime, serialized_engine: buffer) -> tensorrt.tensorrt.ICudaEngine

Invoked with: <tensorrt.tensorrt.Runtime object at 0x7f9867516c30>, None

## ONNX Runtime baseline

ONNX Runtime is the go to inference solution from Microsoft.

The recent 1.10 version of ONNX Runtime (with TensorRT support) is still a bit buggy on transformer models, that is why we use the 1.9.0 version in the measures below.

As before, CPU quantization is dynamic.
Function `
` will set ONNX Runtime to use all cores available and enable any possible optimizations.

In [10]:
optimize_onnx(
    onnx_path="baseline.onnx",
    onnx_optim_model_path="baseline-optimized.onnx",
    fp16=True,
    use_cuda=True,
)

cpu_quantization(input_model_path="baseline-optimized.onnx", output_model_path="baseline-quantized.onnx")

Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 - sequence < 0
Cannot determine if 512 -

Ignore MatMul due to non constant B: /[MatMul_59]
Ignore MatMul due to non constant B: /[MatMul_123]
Ignore MatMul due to non constant B: /[MatMul_188]
Ignore MatMul due to non constant B: /[MatMul_112]
Ignore MatMul due to non constant B: /[MatMul_137]
Ignore MatMul due to non constant B: /[MatMul_93]
Ignore MatMul due to non constant B: /[MatMul_236]
Ignore MatMul due to non constant B: /[MatMul_248]
Ignore MatMul due to non constant B: /[MatMul_265]
Ignore MatMul due to non constant B: /[MatMul_275]
Ignore MatMul due to non constant B: /[MatMul_292]
Ignore MatMul due to non constant B: /[MatMul_356]
Ignore MatMul due to non constant B: /[MatMul_421]
Ignore MatMul due to non constant B: /[MatMul_345]
Ignore MatMul due to non constant B: /[MatMul_370]
Ignore MatMul due to non constant B: /[MatMul_326]
Ignore MatMul due to non constant B: /[MatMul_469]
Ignore MatMul due to non constant B: /[MatMul_481]
Ignore MatMul due to non constant B: /[MatMul_498]
Ignore MatMul due to non constant

In [11]:
labels = [item["label"] for item in encoded_dataset[validation_key]]
data = encoded_dataset[validation_key][0:batch_size]
inputs_onnx: OD[str, np.ndarray] = convert_tensor(data=data, output="np")
for k, v in inputs_onnx.items():
    inputs_onnx[k] = v.astype(np.int64)

model = create_model_for_provider(path="baseline-optimized.onnx", provider_to_use="CUDAExecutionProvider")
output = model.run(None, inputs_onnx)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid Feed Input Name:token_type_ids

In [ ]:
data = encoded_dataset["train"][0:batch_size]
inputs_onnx: OD[str, np.ndarray] = convert_tensor(data=data, output="np")
for k, v in inputs_onnx.items():
    inputs_onnx[k] = v.astype(np.int64)

for provider, model_path, benchmark_name, warmup, nb_inference in [
    ("CUDAExecutionProvider", "baseline.onnx", "ONNX Runtime GPU (FP32)", 10, 100),
    ("CUDAExecutionProvider", "baseline-optimized.onnx", "ONNX Runtime GPU (FP16)", 10, 100),
    ("CPUExecutionProvider", "baseline.onnx", "ONNX Runtime CPU (FP32)", 3, 10),
    ("CPUExecutionProvider", "baseline-optimized.onnx", "ONNX Runtime CPU (FP16)", 3, 10),
    ("CPUExecutionProvider", "baseline-quantized.onnx", "ONNX Runtime CPU (INT-8)", 3, 10),
]:
    model = create_model_for_provider(path=model_path, provider_to_use=provider)
    for _ in range(warmup):
        _ = model.run(None, inputs_onnx)
    time_buffer = []
    for _ in range(nb_inference):
        with track_infer_time(time_buffer):
            _ = model.run(None, inputs_onnx)
    print_timings(name=benchmark_name, timings=time_buffer)
    del model

Measure of the accuracy with ONNX Runtime engine and CUDA provider:

In [18]:
model = create_model_for_provider(path="baseline.onnx", provider_to_use="CUDAExecutionProvider")
infer_ort = lambda tokens: model.run(None, tokens)
measure_accuracy(infer=infer_ort, int64=True)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid Feed Input Name:token_type_ids

In [ ]:
model = create_model_for_provider(path="baseline-optimized.onnx", provider_to_use="CUDAExecutionProvider")
infer_ort = lambda tokens: model.run(None, tokens)
measure_accuracy(infer=infer_ort, int64=True)